In [ ]:
#!pip install transformers datasets evaluate

Необходимые импорты

In [ ]:
import math
import json
import pandas as pd
import re
from datasets import Dataset
from transformers import 
from transformers import 
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

Логинимся в профиль HuggingFace по индивидуальному токену (необходимо для сохранения результатов обучения в хаб, а не локально)

In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


За основу возьмём сберовскую модель, основанную на архитектуре GPT2.

In [ ]:
model_checkpoint = "sberbank-ai/rugpt3small_based_on_gpt2"

In [ ]:
#Вытащим из дампа постов канала только текстовую информацию

posts = []
with open('result.json') as f:
  data = json.load(f)
for post in data['messages']:
  if post['text']:
    text = ''
    for entity in post['text_entities']:
      if entity['type'] == 'plain' or entity['type'] == 'bold':
        text += entity['text']
    posts.append(text)

In [ ]:
#убираем повторяющиеся посты

print(f'Число всех постов: {len(posts)}')
posts = list(set(posts))
print(f'Число постов без дубликатов: {len(posts)}')

Число всех постов: 6016
Число постов без дупликатов: 5179


In [ ]:
#много пробелов подряд замедляют и ухудшают обучение text generation моделей, убираем их

def clean_post(text):
    text = re.sub(' +', ' ', text) 
    return text

In [ ]:
clean_posts = [clean_post(post) for post in posts]

In [ ]:
#создадим датасет формата модуля datasets

df = pd.DataFrame(clean_posts, columns =['text'])
datasets = Dataset.from_pandas(df)

In [ ]:
#разобъём на обучающую и валидационную выборку в соотношении 70/30

datasets = datasets.train_test_split(test_size=0.3)

In [ ]:
#загружаем токенизатор

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'])

In [ ]:
tokenized_datasets = datasets.map(preprocess_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
block_size = 128

In [ ]:
#объединяем все тексты и делим их на части длиной в block_size

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size

    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
#грузим модель

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
#задаём гиперпараметры обучения

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-tat_model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3837
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1440
  Number of trainable parameters = 125231616


Epoch,Training Loss,Validation Loss
1,No log,3.439450
2,3.618400,3.404525
3,3.349300,3.400155


***** Running Evaluation *****
  Num examples = 1629
  Batch size = 8
Saving model checkpoint to rugpt3small_based_on_gpt2-tat_model/checkpoint-500
Configuration saved in rugpt3small_based_on_gpt2-tat_model/checkpoint-500/config.json
Configuration saved in rugpt3small_based_on_gpt2-tat_model/checkpoint-500/generation_config.json
Model weights saved in rugpt3small_based_on_gpt2-tat_model/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1629
  Batch size = 8
Saving model checkpoint to rugpt3small_based_on_gpt2-tat_model/checkpoint-1000
Configuration saved in rugpt3small_based_on_gpt2-tat_model/checkpoint-1000/config.json
Configuration saved in rugpt3small_based_on_gpt2-tat_model/checkpoint-1000/generation_config.json
Model weights saved in rugpt3small_based_on_gpt2-tat_model/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1629
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models

TrainOutput(global_step=1440, training_loss=3.4087157355414495, metrics={'train_runtime': 591.4162, 'train_samples_per_second': 19.463, 'train_steps_per_second': 2.435, 'total_flos': 751933145088000.0, 'train_loss': 3.4087157355414495, 'epoch': 3.0})

In [ ]:
#для подобной задачи перплексия достаточно низкая, модель хорошо предсказывает выборку

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1629
  Batch size = 8


Perplexity: 29.97


In [ ]:
#выгружаем модель и токенизатор

trainer.push_to_hub()
tokenizer.push_to_hub('efromomr/rugpt3small_based_on_gpt2-tat_model')

А теперь протестируем модель

In [ ]:
model = AutoModelForCausalLM.from_pretrained("efromomr/rugpt3small_based_on_gpt2-tat_model")

In [ ]:
inputs = tokenizer('Лингвистика', return_tensors="pt").input_ids

In [4]:
outputs = model.generate(inputs, max_new_tokens=350, do_sample=True, top_k=50, top_p=0.95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Лингвистика и инженерия - 3 \n\nОдно из самых ярких впечатлений в жизни - посещение одной из крупнейших компаний Москвы - группы компаний, занимающихся исследованиями и разработкой. \n\nКороче, ребята решили для себя, что теперь, когда они в Москве, можно будет сделать сайт на русском, английском и французском языках с переводом на любую иностранную валюту. И это не говоря про стоимость. \n\nИ мы начали создавать сайт на сайте, который должен был стоить 440 000 руб..\n\nВсе было очень круто, как только можно в этом сомневаться. И за эти деньги, ребята решили, что наш новый интернет-магазин должен быть максимально удобным для клиентов. И я получил от ребят совершенно бесплатно три полноценных кейса с гарантией - от логотипа до разработки дизайна интерфейсов.\n\nКак только заказчик сделал заказ, а мы с ним рассчитались - у нас появилось еще одно преимущество - мы с ними можем создавать дизайн за 2-3 недели. \n\nИ для меня это просто невероятно круто, потому что теперь есть свой сайт, к